### Import ES & Libs

In [ ]:
import pickle
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [ ]:
# get exist data
res = es.search(index="people", body={"query": {"match_all": {}}})
print(res)

### Index data

In [ ]:
# import file
BF_logs_raw = pd.read_csv("./data/BF_datalab_httpd_all.log", header = None, delimiter = " ")
log_length = BF_logs_raw.shape[0]
BF_logs_raw.head(1)

In [ ]:
from elasticsearch import helpers

def bulkData(length, log_type):
    es = Elasticsearch()
    j = 0
    actions = []
    while (j < length):
        log = BF_logs_raw.loc[j]
        action = {
            "_index": 'log',
            "_type": log_type,
            "_id": j,
            "_source": {
                "timestamp":log[3],
                "action":log[5],
                "status":log[6],
                "respsize":log[7],
                "data":log[9],
                "timed":log[10]
            }
        }
        actions.append(action)
        j += 1
    # use bulk to POST data
    if (len(actions) > 0):
        helpers.bulk(es, actions)
        del actions[0:len(actions)]

In [ ]:
# execute function to store data
bulkData(log_length, 'bf')

#### For other logs

In [ ]:
normal_logs_raw = pd.read_csv("./data/normal.log", header = None, delimiter = " ")
sql_logs_raw = pd.read_csv("./data/sql.log", header = None, delimiter = " ")
xss_logs_raw = pd.read_csv("./data/xss.log", header = None, delimiter = " ",error_bad_lines=False)

log_length_normal = normal_logs_raw.shape[0]
log_length_sql = sql_logs_raw.shape[0]
log_length_xss = xss_logs_raw.shape[0]

In [ ]:
def bulkData(length, log_type):
    es = Elasticsearch()
    j = 0
    actions = []
    # give diffrent separators to diffrent logs
    if log_type == 'bf':
        separator = 0
    if log_type == 'normal':
        separator = 200000
    if log_type == 'sql':
        separator = 400000
    if log_type == 'xss':
        separator = 600000
    while (j < length):
        log = BF_logs_raw.loc[j]
        action = {
            "_index": 'log',
            "_type": log_type,
            "_id": j + separator,
            "_source": {
                "timestamp":log[3],
                "action":log[5],
                "status":log[6],
                "respsize":log[7],
                "data":log[9],
                "timed":log[10]
            }
        }
        actions.append(action)
        j += 1
    # use bulk to POST data
    if (len(actions) > 0):
        helpers.bulk(es, actions)
        del actions[0:len(actions)]

In [ ]:
bulkData(log_length_normal, 'normal')
bulkData(log_length_sql, 'sql')
bulkData(log_length_xss, 'xss')

### Get data 

In [ ]:
es.search(index="log", doc_type='normal',body={"query": {"match_all": {}}})